In [ ]:
# This script implements a simple trading strategy using Exponential Moving Averages (EMA) to generate buy/sell signals.
import yfinance as yf
import pandas as pd
import sys
import matplotlib.pyplot as plt
#Ignore FutureWarnings from pandas
# This is to avoid warnings about future changes in pandas behavior that may not affect the current code
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# 1. Download historical stock data for any company in 2023
xyz = input("Enter the stock ticker symbol (e.g., AAPL for Apple): ").upper()
try:
    data = yf.download(f"{xyz}", start="2023-01-01", end="2023-12-31")
    if not data.empty:
        print("Data found")
    else:
        sys.exit("No data found for the ticker symbol.")

except Exception as e:
    print("Error:", e)

print(f"\nData for {xyz} downloaded successfully.\n")

prices = data['Close'][f'{xyz}'] 


short_span_size = int(input("Enter the window size for short-term EMA: "))
long_span_size = int(input("Enter the window size for long-term EMA: "))

# 2. Calculate short-term and long-term EMAs User input
short_ema = prices.ewm(span=short_span_size).mean()
long_ema = prices.ewm(span=long_span_size).mean()



# 3. Generate trading signals based on EMA crossover
signals = ['Hold']  # no signal on first day

for i in range(1, len(prices)):
    if short_ema.iloc[i] > long_ema.iloc[i] and short_ema.iloc[i-1] <= long_ema.iloc[i-1]:
        signals.append('Buy')
    elif short_ema.iloc[i] < long_ema.iloc[i] and short_ema.iloc[i-1] >= long_ema.iloc[i-1]:
        signals.append('Sell')
    else:
        signals.append('Hold')


# 4. Simulate portfolio with $10,000 cash, buy/sell 1 share per signal
shares = 0
cash = 100000  # starting capital
portfolio_values = []
shares_bought = []
shares_sold = []

for i in range(len(prices)):
    if short_ema[i] > long_ema[i] and shares == 0:
        shares = 100
        cost = shares * prices[i]
        cash -= cost
        shares_bought.append(shares)
        shares_sold.append(0)
    elif short_ema[i] < long_ema[i] and shares > 0:
        cash += shares * prices[i]
        shares_bought.append(0)
        shares_sold.append(shares)
        shares = 0
    else:
        shares_bought.append(0)
        shares_sold.append(0)
    
    portfolio_value = cash + shares * prices[i]
    portfolio_values.append(portfolio_value)

short_col = f'Short EMA ({short_span_size})'
long_col = f'Long EMA ({long_span_size})'


# 5. Create a summary DataFrame with all info
summary = pd.DataFrame({
    'Date': prices.index,
    'Price': prices.values,
    short_col: short_ema.values,
    long_col: long_ema.values,
    'Shares Bought': shares_bought,
    'Shares Sold': shares_sold,
    'Signal': signals,
    'Portfolio Value': portfolio_values
})

summary['Signal'] = 0  # default hold
summary.loc[summary['Shares Bought'] > 0, 'Signal'] = 1  # Buy
summary.loc[summary['Shares Sold'] > 0, 'Signal'] = -1  # Sell
summary['Profit'] = summary['Portfolio Value'].diff().fillna(0)
summary['Trade Profit'] = summary['Profit'].where(summary['Signal'] != 0, 0)
total_profit = summary['Trade Profit'].sum()
buy_trades = summary['Signal'].value_counts().get(1, 0)
sell_trades = summary['Signal'].value_counts().get(-1, 0)

# 6. Calculate total realized profit from trades
# This will sum the profits from each buy-sell pair
trade_profits = []
buy_price = 0
buy_shares = 0
holding = False

for i in range(len(prices)):
    # When buying
    if summary.loc[i, 'Signal'] == 1:
        buy_price = summary.loc[i, 'Price']
        buy_shares = summary.loc[i, 'Shares Bought']
        holding = True

    # When selling and currently holding shares
    elif summary.loc[i, 'Signal'] == -1 and holding:
        sell_price = summary.loc[i, 'Price']
        sell_shares = summary.loc[i, 'Shares Sold']
        profit = (sell_price * sell_shares) - (buy_price * buy_shares)
        trade_profits.append(profit)
        holding = False

# Check if holding shares at the end of the period
if holding:
    # Sell all remaining shares at the last price
    sell_price = summary.loc[len(summary) - 1, 'Price']
    sell_shares = buy_shares  # shares held
    profit = (sell_price * sell_shares) - (buy_price * buy_shares)
    trade_profits.append(profit)
    holding = False

# Sum all trade profits
total_realized_profit = sum(trade_profits)


#Summary of trades

print(summary[summary['Signal'] != 0])

print(f"\nSummary of trades for {xyz}:\n")
print(f"\nTotal Profit: ${total_profit:.2f}")
print(f"Buy Trades: {buy_trades}")
print(f"Sell Trades: {sell_trades}")
print(f"Total Realized Profit from Trades: ${total_realized_profit:.2f}\n")

In [ ]:
#Plotting the results
print(f"\nPlotting the results...{xyz}\n")

plt.figure(figsize=(14, 6))
plt.plot(summary['Date'], summary['Price'], label='Price', alpha=0.5)
plt.plot(summary['Date'], summary[short_col], label=f'{short_span_size}-day EMA')
plt.plot(summary['Date'], summary[long_col], label=f'{long_span_size}-day EMA')



# Plot Buy/Sell signals
buy_signals = summary[summary['Signal'] == 1]
sell_signals = summary[summary['Signal'] == -1]
plt.scatter(buy_signals['Date'], buy_signals['Price'], marker='^', color='green', label='Buy', s=100)
plt.scatter(sell_signals['Date'], sell_signals['Price'], marker='v', color='red', label='Sell', s=100)

plt.legend()
plt.title("EMA Crossover Strategy")
plt.xlabel("Date")
plt.ylabel("Price")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
summary.to_csv('trading_summary_ema.xlsx', index=False)
print("Trading summary saved to 'trading_summary_ema.xlsx'.")